In [1]:
""" 
Steps
1. Download datasets
1. Unzip
1. Convert to shapefile
1. Upload to GCS
1. Ingest into ee.


URL: http://www.nationaalgeoregister.nl/geonetwork/srv/dut/catalog.search#/metadata/%7B25943e6e-bb27-4b7a-b240-150ffeaa582e%7D

Todo: 

earthengine currently only supports shapefile. Rewrite for geopackage / geosjon in future. 


"""


SCRIPT_NAME = "Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01"
OUTPUT_VERSION = 2

GCS_OUTPUT_PATH = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

ec2_input_path = "/volumes/data/{}/input_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_process_path = "/volumes/data/{}/process_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)



In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M08D13 UTC 14:28


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
# Imports
import os
import subprocess

In [4]:
!rm -r {ec2_output_path}
!rm -r {ec2_process_path}
!rm -r {ec2_input_path}

rm: cannot remove '/volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02/': No such file or directory
rm: cannot remove '/volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/process_V02/': No such file or directory
rm: cannot remove '/volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/': No such file or directory


In [5]:
!mkdir -p {ec2_output_path}
!mkdir -p {ec2_process_path}
!mkdir -p {ec2_input_path}

In [6]:
urls = {}
for year in range(2009,2018):
    urls[year] = ("http://geodata.nationaalgeoregister.nl/brpgewaspercelen/extract/{}-definitief/brpgewaspercelen.zip".format(year))

# url for 2018 is slightly different
urls[2018] = ("http://geodata.nationaalgeoregister.nl/brpgewaspercelen/extract/2018-concept/brpgewaspercelen.zip.zip")
    

In [7]:
for year, url in urls.items():
    output_file_path = ec2_input_path + "BRP{}.zip".format(year)
    command = "wget -O {} '{}'".format(output_file_path,url)
    print(command)
    response = subprocess.check_output(command,shell=True)
    

wget -O /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2016.zip 'http://geodata.nationaalgeoregister.nl/brpgewaspercelen/extract/2016-definitief/brpgewaspercelen.zip'
wget -O /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2017.zip 'http://geodata.nationaalgeoregister.nl/brpgewaspercelen/extract/2017-definitief/brpgewaspercelen.zip'
wget -O /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2018.zip 'http://geodata.nationaalgeoregister.nl/brpgewaspercelen/extract/2018-concept/brpgewaspercelen.zip.zip'
wget -O /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2009.zip 'http://geodata.nationaalgeoregister.nl/brpgewaspercelen/extract/2009-definitief/brpgewaspercelen.zip'
wget -O /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2010.zip 'http://geodata.nationaalgeoregister.nl/brpgewaspercelen/extract/2010-definit

In [8]:
files = os.listdir(ec2_input_path)

In [9]:
files

['BRP2012.zip',
 'BRP2010.zip',
 'BRP2014.zip',
 'BRP2013.zip',
 'BRP2009.zip',
 'BRP2018.zip',
 'BRP2017.zip',
 'BRP2015.zip',
 'BRP2011.zip',
 'BRP2016.zip']

In [10]:
for one_file in files:
    command = "unzip -o {}{} -d {}".format(ec2_input_path,one_file,ec2_process_path)
    print(command)
    response = subprocess.check_output(command,shell=True)

unzip -o /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2012.zip -d /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/process_V02/
unzip -o /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2010.zip -d /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/process_V02/
unzip -o /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2014.zip -d /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/process_V02/
unzip -o /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2013.zip -d /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/process_V02/
unzip -o /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/input_V02/BRP2009.zip -d /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/process_V02/
unzip -o /volumes/data/Y2018M08D13_RH_Process

In [11]:
# Inconsistent file name, rename 2014 file

!mv {ec2_process_path}BRP_Gewaspercelen_definitief.gdb {ec2_process_path}BRP_Gewaspercelen_2014.gdb

In [12]:
gdb_files = os.listdir(ec2_process_path)

In [13]:
for gdb_file in gdb_files:
    if gdb_file.endswith('.gdb'):
        filename, extension = gdb_file.split(".")
        command = "/opt/anaconda3/envs/python35/bin/ogr2ogr -f 'ESRI Shapefile' {}{}.shp {}{}".format(ec2_output_path,filename,ec2_process_path,gdb_file)
        print(command)
        response = subprocess.check_output(command,shell=True)

/opt/anaconda3/envs/python35/bin/ogr2ogr -f 'ESRI Shapefile' /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02/BRP_Gewaspercelen_2015.shp /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/process_V02/BRP_Gewaspercelen_2015.gdb
/opt/anaconda3/envs/python35/bin/ogr2ogr -f 'ESRI Shapefile' /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02/BRP_Gewaspercelen_2009.shp /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/process_V02/BRP_Gewaspercelen_2009.gdb
/opt/anaconda3/envs/python35/bin/ogr2ogr -f 'ESRI Shapefile' /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02/BRP_Gewaspercelen_2013.shp /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/process_V02/BRP_Gewaspercelen_2013.gdb
/opt/anaconda3/envs/python35/bin/ogr2ogr -f 'ESRI Shapefile' /volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02/BRP_Gewasp

In [14]:
!gsutil -m cp {ec2_output_path}/* {GCS_OUTPUT_PATH}

Copying file:///volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02//BRP_Gewaspercelen_2009.dbf [Content-Type=application/octet-stream]...
Copying file:///volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02//BRP_Gewaspercelen_2009.shp [Content-Type=application/x-qgis]...
Copying file:///volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02//BRP_Gewaspercelen_2009.prj [Content-Type=application/octet-stream]...
Copying file:///volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02//BRP_Gewaspercelen_2010.dbf [Content-Type=application/octet-stream]...
Copying file:///volumes/data/Y2018M08D13_RH_Process_Basisregistratie_Gewaspercelen_V01/output_V02//BRP_Gewaspercelen_2009.shx [Content-Type=application/x-qgis]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by ed

In [15]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:04:04.062435
